In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

rachananagaraju_cmir_2025_path = kagglehub.dataset_download('rachananagaraju/cmir-2025')

print('Data source import complete.')


In [ ]:
!pip install python-terrier

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.5/167.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 23.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.4 MB/s eta 0:00:00
  Created wheel for chest: filename=chest-0.2.3-py3-none-any.whl size=7611 sha256=35d9236bc7a258abb7f94ab0ca550e55dec98c3d1d31e4bf1d02eb71526764a1
  Stored in directory: /root/.cache/pip/wheels/48/57/13/28831e81239278141f874ee9b7d6d5490a1b1

In [ ]:
import pyterrier as pt
pt.init()

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-assemblies/5.11/terrier-assemblies-5.11-jar-with-dependencies.jar: 100%|██████████| 99.6M/99.6M [00:00<00:00, 254MB/s]


Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-python-helper/0.0.8/terrier-python-helper-0.0.8.jar: 100%|██████████| 36.6k/36.6k [00:00<00:00, 12.8MB/s]

Done



Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipykernel_36/2516926211.py:2: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [ ]:
# accessing files from my drive - HLS

files = ["/kaggle/input/cmir-2025/Baseline_Corpus.trec"]
indexer = pt.TRECCollectionIndexer(f"./index", overwrite = True, stemmer = None, stopwords=None)
indexref = indexer.index(files)
index = pt.IndexFactory.of(indexref)

In [ ]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")
InL2 = pt.BatchRetrieve(index, wmodel="InL2")
Hiemstra_LM = pt.BatchRetrieve(index, wmodel="Hiemstra_LM")

/tmp/ipykernel_36/1013159242.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
/tmp/ipykernel_36/1013159242.py:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25")
/tmp/ipykernel_36/1013159242.py:3: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  pl2 = pt.BatchRetrieve(index, wmodel="PL2")
/tmp/ipykernel_36/1013159242.py:4: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  InL2 = pt.BatchRetrieve(index, wmodel="InL2")
/tmp/ipykernel_36/1013159242.py:5: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) --

In [ ]:
topics = pt.io.read_topics('/kaggle/input/cmir-2025/Train_query_20_Roman.trec')
qrels = pt.io.read_qrels('/kaggle/input/cmir-2025/QRels_Train_20.txt')

In [ ]:
result = pt.Experiment([tf_idf, bm25, pl2, InL2, Hiemstra_LM],
                       topics,
                       qrels,
                       eval_metrics=["map", "recip_rank", "ndcg", "P_5", "P_10"], verbose = True)
print(result)

pt.Experiment: 100%|██████████| 5/5 [00:04<00:00,  1.06system/s]

                       name       map  recip_rank      ndcg   P_5   P_10
0       TerrierRetr(TF_IDF)  0.173530    0.654553  0.393390  0.35  0.250
1         TerrierRetr(BM25)  0.175802    0.662623  0.397553  0.34  0.250
2          TerrierRetr(PL2)  0.165005    0.643877  0.384670  0.33  0.235
3         TerrierRetr(InL2)  0.180464    0.651930  0.403353  0.35  0.275
4  TerrierRetr(Hiemstra_LM)  0.246725    0.801860  0.483592  0.50  0.360


In [ ]:
test_queries = pt.io.read_topics("/kaggle/input/cmir-2025/test_query_30.trec")
test_queries.head()

,qid,query
0,26,hi hyderabad e rapid antigen test kothay kora ...
1,27,hello everyone i need urgent information regar...
2,28,hi all can anyone tell me any adhar center nea...
3,29,amar thakuma kolkata theke aamar kache shift k...
4,30,last time apollo theke rtpcr test koriyechilam...


In [ ]:

import joblib
joblib.dump(bm25, "bm25_model.joblib")


['bm25_model.joblib']

In [ ]:
import pyterrier as pt
import joblib
import pandas as pd

# Initialize PyTerrier
if not pt.started():
    pt.init()

# Load test queries
test_queries = pt.io.read_topics("/kaggle/input/cmir-2025/test_query_30.trec")

# Load trained model
model = joblib.load("/kaggle/working/bm25_model.joblib")

# Generate predictions
results = model.transform(test_queries)

# Format required columns
results = results[["qid", "docno", "rank", "score"]]
results["tag"] = "bm25"

# Save predictions to CSV
results.to_csv("test1_predictions.csv", index=False)


/tmp/ipykernel_36/3184126084.py:6: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


In [ ]:
res_file = bm25.transform(topics)
print(res_file)

      qid   docid   docno  rank      score  \
0       1  106544  106545     0  30.984167   
1       1   78143   78144     1  21.911458   
2       1   33223   33224     2  21.359212   
3       1   60457   60458     3  19.093987   
4       1   71411   71412     4  18.751333   
...    ..     ...     ...   ...        ...   
19995  25   41694   41695   995  16.131297   
19996  25     822     823   996  16.130693   
19997  25   68755   68756   997  16.129350   
19998  25   60021   60022   998  16.127682   
19999  25   65075   65076   999  16.125956   

                                                   query  
0      hyderabad to howrah kono train ki diyeche ba d...  
1      hyderabad to howrah kono train ki diyeche ba d...  
2      hyderabad to howrah kono train ki diyeche ba d...  
3      hyderabad to howrah kono train ki diyeche ba d...  
4      hyderabad to howrah kono train ki diyeche ba d...  
...                                                  ...  
19995  need an information regardi

------------------------------------------------------------------

In [ ]:
files = ["/kaggle/input/cmir-2025/Baseline_Corpus.trec"]
topics = pt.io.read_topics('/kaggle/input/cmir-2025/Train_query_20_Roman.trec')
qrels = pt.io.read_qrels('/kaggle/input/cmir-2025/QRels_Train_20.txt')
test_queries = pt.io.read_topics("/kaggle/input/cmir-2025/test_query_30.trec")


In [ ]:
indexer = pt.TRECCollectionIndexer("/content/index", overwrite = True)  # added overwrite - True as it was giving error
indexref = indexer.index(files)
index = pt.IndexFactory.of(indexref)


11:28:50.479 [main] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (145) - further warnings are suppressed
11:28:58.127 [main] WARN org.terrier.structures.indexing.Indexer -- Indexed 95 empty documents


In [ ]:
bm25       = pt.BatchRetrieve(index, wmodel="BM25")
dph        = pt.BatchRetrieve(index, wmodel="DPH")
pl2        = pt.BatchRetrieve(index, wmodel="PL2")
tfidf      = pt.BatchRetrieve(index, wmodel="TF_IDF")
ql         = pt.BatchRetrieve(index, wmodel="Hiemstra_LM")
dirichlet  = pt.BatchRetrieve(index, wmodel="DirichletLM")
dlh13      = pt.BatchRetrieve(index, wmodel="DLH13")
ifb2       = pt.BatchRetrieve(index, wmodel="IFB2")


/tmp/ipykernel_36/3384829923.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25       = pt.BatchRetrieve(index, wmodel="BM25")
/tmp/ipykernel_36/3384829923.py:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  dph        = pt.BatchRetrieve(index, wmodel="DPH")
/tmp/ipykernel_36/3384829923.py:3: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  pl2        = pt.BatchRetrieve(index, wmodel="PL2")
/tmp/ipykernel_36/3384829923.py:4: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf      = pt.BatchRetrieve(index, wmodel="TF_IDF")
/tmp/ipykernel_36/3384829923.py:5: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.R

In [ ]:
all_models = [tfidf, bm25, pl2, dph, ql, dirichlet, dlh13, ifb2]
model_names = ["TF_IDF", "BM25", "PL2", "DPH", "Hiemstra_LM", "DirichletLM", "DLH13", "IFB2"]

result = pt.Experiment(
    all_models,
    topics,
    qrels,
    eval_metrics=["map", "recip_rank", "ndcg", "P_5", "P_10"],
    names=model_names,
    verbose=True
)

print(result)


pt.Experiment: 100%|██████████| 8/8 [00:05<00:00,  1.40system/s]

          name       map  recip_rank      ndcg   P_5   P_10
0       TF_IDF  0.203710    0.707828  0.435052  0.37  0.275
1         BM25  0.206274    0.711735  0.439723  0.37  0.270
2          PL2  0.198143    0.699313  0.425535  0.36  0.260
3          DPH  0.158178    0.690650  0.407897  0.30  0.235
4  Hiemstra_LM  0.265494    0.831812  0.513845  0.51  0.370
5  DirichletLM  0.212906    0.766310  0.502181  0.39  0.300
6        DLH13  0.199714    0.748438  0.438409  0.36  0.285
7         IFB2  0.227289    0.725407  0.463754  0.41  0.305


In [ ]:

from pyterrier.apply import by_query


fused_model = bm25 + dirichlet + ql

fused_results = fused_model(topics)


result_fused = pt.Experiment(
    [fused_model],
    topics,
    qrels,
    eval_metrics=["map", "recip_rank", "ndcg", "P_5", "P_10"],
    names=["Fused_BM25_Dirichlet_QL"],
    verbose=True
)

print(result_fused)

pt.Experiment: 100%|██████████| 1/1 [00:02<00:00,  2.12s/system]

                      name      map  recip_rank     ndcg   P_5  P_10
0  Fused_BM25_Dirichlet_QL  0.26357    0.827083  0.53679  0.51  0.38


In [ ]:
test_results = fused_model.transform(test_queries)
test_results = test_results[["qid", "docno", "rank", "score"]]
test_results["tag"] = "RRF_Fusion"
test_results.to_csv("test2_predictions.csv", index=False)


---------------------------------------------

In [ ]:
!pip install pyterrier_t5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [ ]:
import pyterrier as pt
import torch
from pyterrier_t5 import MonoT5ReRanker


2025-07-24 11:30:52.430634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753356652.780444      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753356652.883726      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25", verbose=True)
dirichlet = pt.BatchRetrieve(index, wmodel="DirichletLM", verbose=True)
pl2 = pt.BatchRetrieve(index, wmodel="PL2", verbose=True)

/tmp/ipykernel_36/3678423896.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25", verbose=True)
/tmp/ipykernel_36/3678423896.py:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  dirichlet = pt.BatchRetrieve(index, wmodel="DirichletLM", verbose=True)
/tmp/ipykernel_36/3678423896.py:3: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  pl2 = pt.BatchRetrieve(index, wmodel="PL2", verbose=True)


In [ ]:
rm3 = bm25 >> pt.rewrite.RM3(index) >> bm25

In [ ]:
reranker = MonoT5ReRanker("castorini/monot5-base-msmarco")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
class DropInvalidDocs(pt.Transformer):
    def __init__(self, valid_docnos, valid_docids):
        self.valid_docnos = valid_docnos
        self.valid_docids = valid_docids

    def transform(self, input_df):
        print(f"Before filtering: {len(input_df)} rows")
        input_df = input_df.dropna(subset=["docno", "docid"])
        input_df = input_df[
            input_df["docno"].astype(str).str.strip() != ""
        ]
        input_df = input_df[
            input_df["docno"].isin(self.valid_docnos) &
            input_df["docid"].isin(self.valid_docids)
        ]
        print(f"After filtering: {len(input_df)} rows")
        return input_df


In [ ]:
class SafeTextLoader(pt.Transformer):
    def __init__(self, index):
        self.index = index
        self.meta = index.getMetaIndex()

    def transform(self, input_df):
        if "docid" not in input_df.columns:
            raise ValueError("Missing 'docid' in input")

        input_df = input_df.dropna(subset=["docid"])
        input_df["docid"] = input_df["docid"].astype(int)
        docids = input_df["docid"].tolist()

        try:
            texts = self.meta.getItems("text", docids)
            input_df["text"] = texts
        except Exception as e:
            print("Error loading text from MetaIndex:", str(e))
            input_df["text"] = ""

        return input_df


In [ ]:
meta = index.getMetaIndex()
num_docs = index.getCollectionStatistics().getNumberOfDocuments()

valid_docnos = set(str(x) for x in meta.getItems("docno", list(range(num_docs))))
valid_docids = set(range(num_docs))

doc_filter = DropInvalidDocs(valid_docnos, valid_docids)
text_loader = SafeTextLoader(index)
lexical_fusion= bm25+ dirichlet + pl2 +rm3
hybrid_pipeline = (
    bm25 + dirichlet + pl2 + rm3
) >> doc_filter >> text_loader >> reranker


In [ ]:
eval_metrics = ["map", "recip_rank", "ndcg", "P_5", "P_10"]
results = pt.Experiment(
    [bm25, dirichlet, pl2, lexical_fusion, hybrid_pipeline],
    topics,
    qrels,
    eval_metrics=eval_metrics,
    names=["BM25", "Dirichlet", "PL2", "Lexical","HybridFusion+BERT"]
)
print(results)



TerrierRetr(BM25): 100%|██████████| 20/20 [00:01<00:00, 18.15q/s]

TerrierRetr(DirichletLM): 100%|██████████| 20/20 [00:01<00:00, 17.89q/s]

TerrierRetr(BM25): 100%|██████████| 20/20 [00:00<00:00, 31.89q/s]


Before filtering: 44201 rows
After filtering: 20000 rows


monoT5: 100%|██████████| 5000/5000 [1:11:50<00:00,  1.16batches/s]

                name       map  recip_rank      ndcg   P_5   P_10
0               BM25  0.206274    0.711735  0.439723  0.37  0.270
1          Dirichlet  0.212906    0.766310  0.502181  0.39  0.300
2                PL2  0.198143    0.699313  0.425535  0.36  0.260
3            Lexical  0.252724    0.694416  0.517156  0.48  0.360
4  HybridFusion+BERT  0.009483    0.027478  0.186251  0.00  0.005


In [ ]:
test_results = fused_model.transform(test_queries)
test_results = test_results[["qid", "docno", "rank", "score"]]
test_results["tag"] = "Hybrid"
test_results.to_csv("test3_predictions.csv", index=False)
